In [97]:
from pymongo import MongoClient
import pandas as pd
import re
import getpass
import json
import os
import requests
from dotenv import load_dotenv
load_dotenv()
import numpy as np
import functions as f

In [3]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [4]:
db, coll = connectCollection('companies','1MCompanies')

In [98]:
company_locations = list(coll.find({},{"Latitude":1,"Longitude":1}))
len(company_locations)

4499

In [91]:
def starbucksRequestAuthorized(params):
    authToken1 = os.getenv("CLIENT_ID")
    authToken2 = os.getenv("CLIENT_SECRET")
    if not authToken1 or not authToken2:
        raise ValueError("NECESITAS UN TOKEN")
    else:
        print("We have a github token: ", authToken1[0:4])
    params["client_id"] = authToken1
    params["client_secret"] = authToken2
    url = 'https://api.foursquare.com/v2/venues/explore'
    res = requests.get(url, params=params)
    
    return res

In [92]:
def apistarbucks(long,lat):
    location = {"v":"20180323",
               "ll":f'{long},{lat}',
               "query":"Starbucks",
               "limit":1,
               "radius":500}
    data = json.loads(starbucksRequestAuthorized(location).text)
    return data

In [35]:
all_starbucks=[]
for company in company_locations:
    starbucks=dict()
    comp_lat=company['Latitude']
    comp_long=company['Longitude']
    dat=apistarbucks(comp_long,comp_lat)
    resp=dat['response']
    if resp.notnull():
        lt=stb['venue']['location']['lat']
        lg=stb['venue']['location']['lng']
        starbucks['refcomp']=company['_id']
        starbucks['latitude']=lt
        starbucks['longitude']=lg
        starbucks['geoloc_stb']=f.getLocation(lg,lt)
        all_starbucks.append(starbucks)
    break

We have a github token:  31FM


In [82]:
print(dat['response']['groups'][0]['items'][1]['venue']['location']['lat'])
dat['response']['groups'][0]['items'][1]['venue']['location']['lng']

40.72689


-74.00026

In [58]:
len(dat['response']['groups'][0]['items'])

4